In [19]:
import numpy as np 
import pandas as pd
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

titanic_train =pd.read_csv("/workspaces/titanic_dataset/train.csv")
titanic_train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
titanic_test = pd.read_csv("/workspaces/titanic_dataset/test.csv")
titanic_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [11]:

#titanic_train.info()
titanic_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [40]:
# ...existing code...
import importlib
import numpy as np
import pandas as pd
import xgboost as xgb          # keep as module name
import scipy.sparse as sp
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

# read data
titanic_train = pd.read_csv("/workspaces/titanic_dataset/train.csv")
titanic_test = pd.read_csv("/workspaces/titanic_dataset/test.csv")

# prepare features/target (drop obvious non-features)
X = titanic_train.drop(columns=["Survived", "Name", "Ticket", "Cabin", "PassengerId"], errors="ignore")
y = titanic_train["Survived"]

# split first to avoid leakage
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# feature lists (adjust if needed)
num_cols = ["Age", "Fare", "SibSp", "Parch"]
pclass_col = ["Pclass"]               # ordinal
cat_cols = ["Sex", "Embarked"]        # nominal -> one-hot

# numeric pipeline
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# ordinal pipeline for Pclass
pclass_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ord", OrdinalEncoder(categories=[[1, 2, 3]]))
])

# categorical pipeline - use sparse_output=False (compatible with newer sklearn)
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("pclass", pclass_pipeline, pclass_col),
    ("cat", cat_pipeline, cat_cols),
], remainder="drop")

# pipeline for cross-validation (no early stopping here)
full_pipeline = Pipeline([
    ("preproc", preprocessor),
    ("clf", XGBClassifier(use_label_encoder=False, eval_metric="logloss",
                          n_estimators=200, learning_rate=0.1, random_state=42))
])

# cross-validate on training split


In [42]:
# ...existing code...
import importlib
import numpy as np
import pandas as pd
import xgboost as xgb    # ensure module name is not shadowed
import scipy.sparse as sp
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

# reload xgboost module to avoid accidental shadowing from earlier cells
importlib.reload(xgb)

# read data (if not already read)
titanic_train = pd.read_csv("/workspaces/titanic_dataset/train.csv")
titanic_test  = pd.read_csv("/workspaces/titanic_dataset/test.csv")

# prepare features/target (drop non-features)
X = titanic_train.drop(columns=["Survived", "Name", "Ticket", "Cabin", "PassengerId"], errors="ignore")
y = titanic_train["Survived"]

# split first to avoid leakage
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# feature lists
num_cols   = ["Age", "Fare", "SibSp", "Parch"]
pclass_col = ["Pclass"]            # ordinal
cat_cols   = ["Sex", "Embarked"]   # nominal

# pipelines
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

pclass_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ord", OrdinalEncoder(categories=[[1, 2, 3]])),
])

# use sparse_output=False to be compatible with sklearn versions that expect it
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
])

preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("pclass", pclass_pipeline, pclass_col),
    ("cat", cat_pipeline, cat_cols),
], remainder="drop")

# pipeline for CV (no early stopping here)
full_pipeline = Pipeline([
    ("preproc", preprocessor),
    ("clf", XGBClassifier(use_label_encoder=False, eval_metric="logloss",
                          n_estimators=200, learning_rate=0.1, random_state=42)),
])

# cross-validate on training split
cv_scores = cross_val_score(full_pipeline, X_train, y_train, cv=5, scoring="accuracy")
print("CV accuracy (5-fold) mean:", cv_scores.mean(), "std:", cv_scores.std())

# fit preprocessor and prepare arrays for xgboost native API (DMatrix)
X_train_t = preprocessor.fit_transform(X_train)
X_val_t   = preprocessor.transform(X_val)

# convert sparse -> dense if necessary (DMatrix prefers dense numpy)
if sp.issparse(X_train_t):
    X_train_t = X_train_t.toarray()
if sp.issparse(X_val_t):
    X_val_t = X_val_t.toarray()

# ensure numpy dtype
X_train_t = np.asarray(X_train_t, dtype=np.float32)
X_val_t   = np.asarray(X_val_t, dtype=np.float32)
y_train_a = y_train.to_numpy()
y_val_a   = y_val.to_numpy()

# use native xgboost.train for early stopping (avoids wrapper inconsistencies)
dtrain = xgb.DMatrix(X_train_t, label=y_train_a)
dval   = xgb.DMatrix(X_val_t, label=y_val_a)

params = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "eta": 0.05,   # learning_rate
    "seed": 42,
}

bst = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=[(dval, "validation")],
    early_stopping_rounds=20,
    verbose_eval=False,
)

# predict using best iteration (handle API differences)
best_it = getattr(bst, "best_iteration", None)
try:
    if best_it is not None:
        probs = bst.predict(dval, ntree_limit=best_it + 1)
    else:
        probs = bst.predict(dval)
except TypeError:
    # newer xgboost may use iteration_range
    if best_it is not None:
        probs = bst.predict(dval, iteration_range=(0, best_it + 1))
    else:
        probs = bst.predict(dval)

val_preds = (probs > 0.5).astype(int)
print("Validation accuracy (early-stopped):", accuracy_score(y_val_a, val_preds))
# ...existing code...

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [13:45:50] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [13:45:50] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [13:45:50] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [13:45:50] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/python/3.12.1/lib/python3.12/site

CV accuracy (5-fold) mean: 0.8020782034866543 std: 0.03711818992675806
Validation accuracy (early-stopped): 0.7877094972067039


In [1]:
# ...existing code...
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay, classification_report

sns.set(style="whitegrid")

# 1) training/validation logloss over boosting rounds (if available)
evals_result = {}
try:
    evals_result = bst.evals_result()
except Exception:
    pass

if evals_result:
    # find first valid eval set and metric
    plt.figure(figsize=(6,4))
    # evals_result structure: { "validation": {"logloss": [...], ...}, ... }
    for data_name, metrics in evals_result.items():
        for metric_name, values in metrics.items():
            plt.plot(values, label=f"{data_name}-{metric_name}")
    plt.xlabel("Boosting round")
    plt.ylabel("Metric")
    plt.title("Training/Validation metrics per boosting round")
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("No evals_result available to plot training history.")

# 2) ROC curve + AUC
try:
    fpr, tpr, _ = roc_curve(y_val_a, probs)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(6,5))
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
    plt.plot([0,1],[0,1], linestyle="--", color="gray")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC curve (validation)")
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.show()
except Exception as e:
    print("ROC plot failed:", e)

# 3) Confusion matrix + classification report
try:
    cm = confusion_matrix(y_val_a, val_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
    fig, ax = plt.subplots(figsize=(4,4))
    disp.plot(ax=ax, cmap="Blues", colorbar=False)
    plt.title("Confusion Matrix (validation)")
    plt.tight_layout()
    plt.show()

    print("Classification report (validation):")
    print(classification_report(y_val_a, val_preds, digits=4))
except Exception as e:
    print("Confusion matrix failed:", e)

# 4) Feature importances mapped to preprocessor output feature names
try:
    # build feature names from fitted preprocessor
    feature_names = []
    # numeric
    feature_names.extend(num_cols)
    # ordinal
    feature_names.extend(pclass_col)
    # one-hot: get fitted onehot encoder categories
    onehot = preprocessor.named_transformers_["cat"].named_steps["onehot"]
    cat_input_col = cat_cols[0] if len(cat_cols) == 1 else None
    # build names for all categorical columns in cat_cols
    # We assume OneHotEncoder was applied to all cat_cols together
    if hasattr(onehot, "categories_"):
        cat_feature_names = []
        # If multiple categorical input columns, categories_ is list in same order
        for col, cats in zip(cat_cols, onehot.categories_):
            cat_feature_names.extend([f"{col}_{str(c)}" for c in cats])
        feature_names.extend(cat_feature_names)
    # get importance dict from booster
    imp = bst.get_score(importance_type="gain")  # fallback weight/cover if needed
    if not imp:
        imp = bst.get_score(importance_type="weight")
    # convert keys 'f0'.. to indices and map to names
    imp_map = {}
    for k, v in imp.items():
        idx = int(k.lstrip("f"))
        name = feature_names[idx] if idx < len(feature_names) else f"f{idx}"
        imp_map[name] = v
    # prepare dataframe for plotting
    import pandas as pd
    fi = pd.DataFrame.from_dict(imp_map, orient="index", columns=["importance"])
    fi = fi.sort_values("importance", ascending=True)
    if not fi.empty:
        plt.figure(figsize=(6, max(4, 0.25 * len(fi))))
        fi["importance"].plot(kind="barh", color="C1")
        plt.title("Feature importance (gain)")
        plt.xlabel("Importance")
        plt.tight_layout()
        plt.show()
    else:
        print("No feature importances available from booster.")
except Exception as e:
    print("Feature importance plotting failed:", e)
# ...existing code...

No evals_result available to plot training history.
ROC plot failed: name 'y_val_a' is not defined
Confusion matrix failed: name 'y_val_a' is not defined
Feature importance plotting failed: name 'num_cols' is not defined
